In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
df = pd.read_csv(r"data/laptop_info.csv")

In [21]:
useles_columns = ['№', 'Модель (полное название)', 'Модель процессора', 'Год выпуска процессора', 'Модель встроенной видеокарты', 'Тип (марка) куллера материнской платы', 'Браузер для тестов','Исполнитель']
df = df.drop(labels = useles_columns,axis = 1)

In [22]:
num_features = ['Количество ядер', 'Количество логических процессоров (потоков)', 'Тактовая частота процессора (ГГц)','HDD','SSD' , 'Объем видеопамяти (Гб)', 'Объем видеопамяти (Гб).1']
cat_features = ['Вид графического ускорителя (Дискретный / Встроенный)', 'Тип видеопамяти', 'Размер оперативной памяти (Гб)']

In [23]:
obj_df = df[num_features].copy()
obj_df['Объем видеопамяти (Гб).1'] = pd.to_numeric(obj_df['Объем видеопамяти (Гб).1'], errors='coerce')
obj_df = obj_df.fillna({"Объем видеопамяти (Гб)": 0,
                       "Объем видеопамяти (Гб).1": 0})

In [24]:
df[num_features] = obj_df

In [25]:
def rename_cols(old_name, one_hot):
    new_cols = {}
    for column in one_hot:
        new_cols[column] = old_name + ": " + column
    return one_hot.rename(columns=new_cols)

In [26]:
graph_acc = pd.get_dummies(df['Вид графического ускорителя (Дискретный / Встроенный)'])
graph_acc.columns = graph_acc.columns.str.strip()
graph_acc.loc[graph_acc['дискретный и встроенный'] == 1, ['встроенный', 'дискретный']] = 1
graph_acc = graph_acc.drop('дискретный и встроенный', axis=1)
graph_acc = rename_cols('Вид графического ускорителя', graph_acc)
    

In [27]:
video_mem = pd.get_dummies(df['Тип видеопамяти'])
video_mem = rename_cols('Тип видеопамяти', video_mem)

In [28]:
RAM_size = (df['Размер оперативной памяти (Гб)'] // 8 > 0).astype(int)

In [29]:
def add_cols(df, add_df):
    for column in add_df.columns:
        df[column] = add_df[column]

In [ ]:
df = df.drop(cat_features, axis=1)
add_cols(df, graph_acc)
add_cols(df, video_mem)
df['Оперативная память > 8'] = RAM_size

In [32]:
benchmark_columns = ['Учебный', 'Развлекательный', 'Эталон']
benchmark_df = df[benchmark_columns].copy()
benchmark_df = benchmark_df.fillna(0)
benchmark_df['sum'] = benchmark_df[benchmark_columns].sum(axis=1)
df = df.drop(['Учебный', 'Развлекательный', 'Эталон'], axis=1)

In [33]:
df['benchmark'] = benchmark_df['sum']

In [36]:
df = df.drop('Модель дискретной видеокарты', axis=1)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 22 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Количество ядер                              23 non-null     int64  
 1   Количество логических процессоров (потоков)  23 non-null     int64  
 2   Тактовая частота процессора (ГГц)            23 non-null     float64
 3   Максимальная тактовая частота (ГГц)          23 non-null     float64
 4   Объем кэша L2 процессора (Кб)                23 non-null     int64  
 5   Объем кэша L3 процессора (Кб)                23 non-null     int64  
 6   Частота оперативной памяти (МГц)             23 non-null     int64  
 7   HDD                                          23 non-null     int64  
 8   SSD                                          23 non-null     int64  
 9   Объем видеопамяти (Гб)                       23 non-null     float64
 10  Объе